In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


In [2]:
df = pd.read_csv('email_data.csv', low_memory=False)

In [ ]:
`# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Train logistic regression
scaler = StandardScaler()  # Standardize features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Evaluate model
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f'Logistic Regression Accuracy: {accuracy:.4f}')
# EDA
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Read data from Parquet file.
df = pd.read_parquet('https://data434.s3.us-east-2.amazonaws.com/Phishing_Email.parquet')

print(df.info())
print(df.isnull().sum())

# Summary Statistics
print(df.describe())

# Target Variable Distribution
sns.countplot(x='Email Type', data=df)
plt.title('Distribution of Email Types')
plt.show()

import matplotlib.pyplot as plt

# Calculate email text lengths
df['email_text_length'] = df['Email Text'].str.len()

# Remove outliers (e.g., top and bottom 1%)
lower_bound = df['email_text_length'].quantile(0.01)
upper_bound = df['email_text_length'].quantile(0.99)
df_filtered = df[(df['email_text_length'] >= lower_bound) & (df['email_text_length'] <= upper_bound)]

# Create the boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x='Email Type', y='email_text_length', data=df_filtered)
plt.title('Email Text Length by Email Type (Outliers Removed)')
plt.xlabel('Email Type')
plt.ylabel('Email Text Length')
plt.show()

#Further analysis (optional):
# Calculate and print descriptive statistics for email text length by email type.
print(df_filtered.groupby('Email Type')['email_text_length'].describe())
